# import 

In [2]:
%autosave 300
import os
import pprint
import tempfile
import tensorflow as tf
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import tempfile

from typing import Dict, Text

Autosaving every 300 seconds


# read data

In [3]:
ratings = tfds.load("movielens/100k-ratings",data_dir= '/data/jyu29/data/recommendation_data/tf_dataset', split="train")


In [4]:
for x in ratings.take(2).as_numpy_iterator():pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'bucketized_user_age': 25.0,
 'movie_genres': array([ 4, 14]),
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'raw_user_age': 32.0,
 'timestamp': 875654590,
 'user_gender': True,
 'user_id': b'92',
 'user_occupation_label': 5,
 'user_occupation_text': b'entertainment',
 'user_rating': 2.0,
 'user_zip_code': b'80525'}


In [10]:
tfds.as_dataframe(ratings.take(2)).T

,0,1
movie_title,"b""One Flew Over the Cuckoo's Nest (1975)""",b'Strictly Ballroom (1992)'
user_id,b'138',b'92'
user_rating,4.0,2.0


## data processing

In [9]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [11]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie titles present in the data.

This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: this allows us to look up the corresponding embeddings in our embedding tables.

In [12]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [15]:
unique_movie_titles[:5]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)'],
      dtype=object)

In [16]:
unique_user_ids[:5]

array([b'1', b'10', b'100', b'101', b'102'], dtype=object)

# ranking model

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures.

A model composed of multiple stacked dense layers is a relatively common architecture for ranking tasks. We can implement it as follows:

In [14]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

    # Compute embeddings for users.
        self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
        self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
        self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

    def call(self, inputs):

        user_id, movie_title = inputs

        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_title)

        return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

This model takes user ids and movie titles, and outputs a predicted rating:



In [20]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01992431]], dtype=float32)>

## loss and metrics

The next component is the loss used to train our model. TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the Ranking task object: a convenience wrapper that bundles together the loss function and metric computation.

We'll use it together with the MeanSquaredError Keras loss in order to predict the ratings.




In [21]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

The task itself is a Keras layer that takes true and predicted as arguments, and returns the computed loss. We'll use that to implement the model's training loop.

## full model

We can now put it all together into a model. TFRS exposes a base model class (`tfrs.models.Model`) which streamlines bulding models: all we need to do is to set up the components in the `__init__` method, and implement the compute_loss method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.




In [22]:
class MovielensModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss = tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

    def call(self, features: Dict[str, tf.Tensor]):
        return self.ranking_model(
        (features["user_id"], features["movie_title"]))

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
        labels = features.pop("user_rating")

        rating_predictions = self(features)

    # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

## fitting

In [23]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [24]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [25]:
model.fit(cached_train, epochs=3)


Epoch 1/3
10/10 [==============================] - 2s 27ms/step - root_mean_squared_error: 2.4902 - loss: 5.6460 - regularization_loss: 0.0000e+00 - total_loss: 5.6460
Epoch 2/3
10/10 [==============================] - 0s 9ms/step - root_mean_squared_error: 1.1228 - loss: 1.2584 - regularization_loss: 0.0000e+00 - total_loss: 1.2584
Epoch 3/3
10/10 [==============================] - 0s 8ms/step - root_mean_squared_error: 1.1174 - loss: 1.2460 - regularization_loss: 0.0000e+00 - total_loss: 1.2460


In [26]:
model.evaluate(cached_test, return_dict=True)


5/5 [==============================] - 1s 12ms/step - root_mean_squared_error: 1.1123 - loss: 1.2317 - regularization_loss: 0.0000e+00 - total_loss: 1.2317


{'root_mean_squared_error': 1.1122585535049438,
 'loss': 1.2079155445098877,
 'regularization_loss': 0,
 'total_loss': 1.2079155445098877}

## Testing the ranking model

Now we can test the ranking model by computing predictions for a set of movies and then rank these movies based on the predictions:

In [31]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
    test_ratings[movie_title] = model({
      "user_id": np.array(["10"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), 
                           key=lambda x: x[1], 
                           reverse=True):
    print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.6254692]]
Dances with Wolves (1990): [[3.5823646]]
Speed (1994): [[3.545468]]


## serving

In [ ]:
tf.saved_model.save(model, "export")


In [ ]:
loaded = tf.saved_model.load("export")

loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

The model above gives us a decent start towards building a ranking system.

Of course, making a practical ranking system requires much more effort.

In most cases, a ranking model can be substantially improved by using more features rather than just user and candidate identifiers. To see how to do that, have a look at the side features tutorial.

A careful understanding of the objectives worth optimizing is also necessary. To get started on building a recommender that optimizes multiple objectives, have a look at our multitask tutorial.